In [1]:
import json
import time
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from twelvedata import TDClient

In [3]:
API_KEY = "7af2850544354c299f71820a47f5d5e8"
td = TDClient(apikey=API_KEY)

def fetch_exchange_rates(currencies):
    """
    Fetch exchange rates for only one direction and compute inverses.
    """
    exchange_rates = {}

    pairs = [] # currency pairs to fetch
    for i in range(len(currencies)):
        for j in range(i + 1, len(currencies)):  # bottom left triangle of nxn matrix
            pairs.append(f"{currencies[i]}/{currencies[j]}")

    batch_size = 8  # TwelveData free plan rate limit
    num_batches = len(pairs) // batch_size + (1 if len(pairs) % batch_size != 0 else 0) # batching requests

    print(f"Fetching {len(pairs)} forex pairs in {num_batches} batches (1 batch per minute)...")

    for i in range(0, len(pairs), batch_size):
        
        batch = pairs[i : i + batch_size]
        print(f"Fetching batch {i // batch_size + 1}/{num_batches}: {batch}")

        try:
        
            prices = td.price(symbol=batch).as_json()

            for pair, data in prices.items():
                base, quote = pair.split("/")
                rate = float(data["price"])

                # adjusting JPY-based quotes
                if base == "JPY":
                    rate /= 100 
                if quote == "JPY":
                    rate *= 100

                if base not in exchange_rates:
                    exchange_rates[base] = {}
                if quote not in exchange_rates:
                    exchange_rates[quote] = {}
                
                exchange_rates[base][quote] = rate     # saving direct exchange rate
                exchange_rates[quote][base] = 1 / rate # compute and save the inverse rate

            # saving progress in case of failures
            with open("forex_rates.json", "w") as f:
                json.dump(exchange_rates, f, indent=4)

            print("Batch saved successfully.")

        except Exception as e:
            print(f"Error fetching batch: {e}")

        # wait to avoid exceeding the rate limit
        if i + batch_size < len(pairs):
            print("Waiting 60 seconds before next batch...")
            time.sleep(60)

    print("All exchange rates successfully retrieved and saved.")
    return exchange_rates


In [ ]:
currencies = ["USD", "EUR", "GBP", "CHF", "CNY", "NOK", "DKK", "SEK", "AUD", "NZD", "CAD"]
exchange_rates = fetch_exchange_rates(currencies)

Fetching 55 forex pairs in 7 batches (1 batch per minute)...
Fetching batch 1/7: ['USD/EUR', 'USD/GBP', 'USD/CHF', 'USD/CNY', 'USD/NOK', 'USD/DKK', 'USD/SEK', 'USD/AUD']
Batch saved successfully.
Waiting 60 seconds before next batch...
Fetching batch 2/7: ['USD/NZD', 'USD/CAD', 'EUR/GBP', 'EUR/CHF', 'EUR/CNY', 'EUR/NOK', 'EUR/DKK', 'EUR/SEK']
Batch saved successfully.
Waiting 60 seconds before next batch...
Fetching batch 3/7: ['EUR/AUD', 'EUR/NZD', 'EUR/CAD', 'GBP/CHF', 'GBP/CNY', 'GBP/NOK', 'GBP/DKK', 'GBP/SEK']
Batch saved successfully.
Waiting 60 seconds before next batch...


In [ ]:
print(exchange_rates)

In [ ]:
def load_exchange_rates(filename="forex_rates.json"):
    """
    Load exchange rates from a saved JSON file.
    """
    with open(filename, "r") as f:
        return json.load(f)

def construct_graph(exchange_rates):
    """
    Construct a forex graph with -log(exchange rates) as edge weights.
    """
    graph = {}
    
    for base in exchange_rates:
        graph[base] = {}
        for quote, rate in exchange_rates[base].items():
            graph[base][quote] = -np.log(rate)  # negative log transform

    return graph, exchange_rates  # return both graph and original rates

def visualize_fxr_graph(graph, exchange_rates):
    """
    Visualize the fx rates graph using NetworkX.
    """
    G = nx.DiGraph()

    for base in graph:
        for quote, weight in graph[base].items():
            G.add_edge(base, quote, weight=exchange_rates[base][quote])  # using original rates for clarity

    plt.figure(figsize=(10, 6))
    pos = nx.spring_layout(G, seed=42)
    edge_labels = {(u, v): f"{d['weight']:.4f}" for u, v, d in G.edges(data=True)}

    nx.draw(G, pos, with_labels=True, node_color="lightblue", edge_color="gray", font_size=10, node_size=3000)
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)

    plt.title("Fx Rates Graph")
    plt.show()

def bellman_ford(graph, exchange_rates, start_currency):
    """
    Runs Bellman-Ford to detect arbitrage cycles.
    
    graph: Forex graph where edges are -log(exchange rates).
    exchange_rates: Original exchange rates to be used for profit calculation.
    start_currency: Arbitrary starting node.
    Arbitrage cycle and its profit value if found, else None.
    """
    
    currencies = list(graph.keys())
    dist = {currency: float("inf") for currency in currencies}
    parent = {currency: None for currency in currencies}
    dist[start_currency] = 0

    # Relax edges |V|-1 times
    for _ in range(len(currencies) - 1):
        for base in graph:
            for quote, weight in graph[base].items():
                if dist[base] + weight < dist[quote]:
                    dist[quote] = dist[base] + weight
                    parent[quote] = base

    # Check for negative cycles
    for base in graph:
        for quote, weight in graph[base].items():
            if dist[base] + weight < dist[quote]:
                
                # negative cycle
                arbitrage_cycle = []
                visited = set()
                curr = quote

                for _ in range(len(currencies)):
                    curr = parent[curr] # tracing back via parents

                cycle_start = curr
                while True:
                    arbitrage_cycle.append(curr)
                    curr = parent[curr]
                    if curr == cycle_start:  # complete cycle
                        break
                
                arbitrage_cycle.append(cycle_start)  # Close the cycle
                arbitrage_cycle.reverse()

                # Calculate arbitrage profit using correct rates
                profit = compute_arbitrage_profit(exchange_rates, arbitrage_cycle)

                print("Arbitrage Opportunity Found!")
                print("Arbitrage Path:", " → ".join(arbitrage_cycle))
                print(f"Starting with $1 in {arbitrage_cycle[0]}, you end with ${profit:.6f}")

                return arbitrage_cycle, profit

    print("No arbitrage opportunity detected.")
    return None, None

def compute_arbitrage_profit(exchange_rates, cycle):
    """
    Compute profit from following an arbitrage cycle, starting with $1.
    """
    amount = 1.0
    print("\nExchange steps for profit calculation:")

    for i in range(len(cycle) - 1):
        base, quote = cycle[i], cycle[i + 1]
        rate = exchange_rates[base][quote]

        print(f"{base} → {quote}: Rate = {rate:.6f}, Amount = {amount:.6f} → {amount * rate:.6f}")
        amount *= rate  # apply rate

    return amount


In [ ]:
# Load exchange rates
exchange_rates = load_exchange_rates()

# Construct the forex graph and return both the graph and original rates
forex_graph, original_rates = construct_graph(exchange_rates)

# Visualize the graph
visualize_graph(forex_graph, original_rates)

# Run Bellman-Ford on an arbitrary starting currency
start_currency = "USD"  # Can be any currency in the dataset
arbitrage_cycle, profit = bellman_ford(forex_graph, original_rates, start_currency)